In [ ]:
import yfinance as yf
# import argparse
import atexit
import datetime
import logging
import schedule
import time
import json

In [ ]:
logger_format = '%(asctime)-15s %(message)s'
logging.basicConfig(format=logger_format)
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [ ]:
from kafka import KafkaProducer
from kafka.errors import KafkaError, KafkaTimeoutError

In [ ]:
topic_name = 'stockTopic'
kafka_broker = '127.0.0.1:9092'

In [ ]:
def fetch_symbol_price(symbol,producer):
  '''
  function to fetch price of the stock and push it to the producer instance
  of kafka
  :param symbol: symbol of the stock
  :param producer: instance of the kafka producer
  :return: None
  '''

  logger.info(f'Fetching stock price of {symbol}')
  try:
    try:
        price=yf.Ticker(symbol).info['currentPrice']
    except Exception as e:
        price=yf.Ticker(symbol).info['regularMarketPrice']
    timestamp=datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%dT%H:%M:%SZ')
    payload={
        'StockSymbol':symbol,
        'Price':price,
        'Time':timestamp
    }
    producer.send(topic=topic_name, value=payload)
    logger.info(f'Received stock price of {symbol} at {timestamp} : {price}')
  except KafkaError as ke:
    logger.warning(f'Kafka Error {ke}')
  except Exception as e:
    logger.warning(f'Failed to fetch stock price of {symbol} caused by {e}')
  return payload

In [ ]:
def on_shutdown(producer):
  '''
  shutdown hook to be called before the shutdown
  :param producer: instance of a kafka producer
  :return: None
  '''
  try:
    logger.info('Flushing pending messages to kafka, timeout is set to 10s')
    producer.flush(10) 
    logger.info('Finish flushing pending messages to kafka')
  finally:
      try:
        logger.info('Closing the Kafka connection')
        producer.close(10)
      except Exception as e:
        log_exception('Failed to close kafka connection',e)

In [ ]:
if __name__=='__main__':
    symbol='GC=F'
    producer=KafkaProducer(
        bootstrap_servers=kafka_broker,
        value_serializer=lambda x: json.dumps(x).encode('utf-8')
    )
    schedule.every(1).second.do(fetch_symbol_price,symbol,producer)
    atexit.register(on_shutdown,producer)
    while True:
        schedule.run_pending()
#         time.sleep(20)